<a href="https://colab.research.google.com/github/ranjith13119/BI-LSTM-FakeNews-Classification/blob/main/BI_LSTM_FakeNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/NLP - LSTM- SPAM/Fake-News-SPAM/train.csv")

In [ ]:
df_data = df.dropna()

In [ ]:
X = df_data.drop('label',axis=1)
y = df_data.label

In [ ]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Bidirectional, Dense, LSTM

In [ ]:
message = X.copy()

In [ ]:
message.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(message)):
    review = re.sub('[^a-zA-Z]', ' ', message['title'][i])
    review = review.lower()
    review = review.split()    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
voc_size=5000

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(len(embedded_docs[0]))

20


In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 14s 53ms/step - loss: 0.3083 - accuracy: 0.8536 - val_loss: 0.1974 - val_accuracy: 0.9173
Epoch 2/10
192/192 [==============================] - 9s 48ms/step - loss: 0.1390 - accuracy: 0.9452 - val_loss: 0.1905 - val_accuracy: 0.9238
Epoch 3/10
192/192 [==============================] - 9s 48ms/step - loss: 0.0915 - accuracy: 0.9676 - val_loss: 0.2332 - val_accuracy: 0.9167
Epoch 4/10
192/192 [==============================] - 9s 48ms/step - loss: 0.0555 - accuracy: 0.9821 - val_loss: 0.2729 - val_accuracy: 0.9190
Epoch 5/10
192/192 [==============================] - 9s 47ms/step - loss: 0.0362 - accuracy: 0.9887 - val_loss: 0.3362 - val_accuracy: 0.9123
Epoch 6/10
192/192 [==============================] - 9s 47ms/step - loss: 0.0287 - accuracy: 0.9899 - val_loss: 0.4569 - val_accuracy: 0.9100
Epoch 7/10
192/192 [==============================] - 9s 47ms/step - loss: 0.0214 - accuracy: 0.9935 - val_loss: 0.4140 - val_accuracy: 0.913

In [ ]:
y_pred=model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3096,  323],
       [ 222, 2394]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9096934548467275

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.88      0.92      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

